In [1]:
import nltk
import spacy
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json

# Download necessary NLTK data
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('punkt')

# Load spaCy model for lemmatization
nlp = spacy.load('en_core_web_sm')

# Preprocessing functions
def preprocess_text(text):
    # Step 1: Tokenization
    # Convert to lowercase and tokenize
    tokens = nltk.word_tokenize(text.lower())

    # Step 2: Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Step 3: Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Step 4: Lemmatization
    doc = nlp(" ".join(tokens))
    tokens = [token.lemma_ for token in doc]

    # Return the processed text as a string
    return " ".join(tokens)

# Chatbot setup
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Chatbot response generation
def get_bot_response(user_message):
    preprocessed_message = preprocess_text(user_message)
    input_ids = tokenizer.encode(preprocessed_message + tokenizer.eos_token, return_tensors="pt")
    response_ids = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# Specify models for pipelines
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)

# Sentiment analysis
def analyze_sentiment(user_message):
    preprocessed_message = preprocess_text(user_message)
    return sentiment_pipeline(preprocessed_message)[0]

# Named Entity Recognition (modified to use raw input)
def perform_ner(user_message):
    return ner_pipeline(user_message)

# Chat history
chat_history = []

# Save chat history to a JSON file
def save_chat_history_to_json(filename="chat_history.json"):
    with open(filename, "w") as file:
        json.dump(chat_history, file, indent=4)
    print(f"Chat history saved to {filename}")

# Main loop for dynamic user input
if __name__ == "__main__":
    print("Welcome to DataPal-Bot! Type 'exit' to end the chat.\n")
    
    while True:
        user_message = input("User: ")  # Get input from the user
        
        if user_message.lower() == "exit":  # Exit condition
            print("\nSaving chat history...")
            save_chat_history_to_json()  # Save chat history to a file
            print("Goodbye!")
            break
        
        # Add the user's message to the chat history
        chat_history.append({"User": user_message})
        
        # Generate bot response
        bot_response = get_bot_response(user_message)
        print("DataPal-Bot:", bot_response)
        
        # Add the bot's response to the chat history
        chat_history.append({"Bot": bot_response})

        # Optional: Show sentiment analysis and NER for the user's message
        sentiment = analyze_sentiment(user_message)
        print(f"Sentiment Analysis: {sentiment}")
        
        entities = perform_ner(user_message)
        print(f"Named Entities: {entities}")


ModuleNotFoundError: No module named 'spacy'